<a href="https://colab.research.google.com/github/edrinthomas24/Hackathon-Ai/blob/main/crowd_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics opencv-python numpy flask streamlit matplotlib twilio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 913.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # 'n' stands for Nano model (lightweight)

# Print model details
print("YOLO model loaded successfully!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 19.8MB/s]


YOLO model loaded successfully!


In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow # Import the necessary function

# Load YOLOv8 model (Pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")

# Load the pre-recorded video
video_path = "/content/WhatsApp Video 2025-02-13 at 10.44.43_248db3db.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)

    # Extract person detections
    people_count = sum(1 for obj in results[0].boxes.data if int(obj[5]) == 0)  # COCO class 0 = 'person'

    # Determine zone status
    if people_count < 5:
        zone_status = "Green (Safe)"
        color = (0, 255, 0)
    elif people_count < 15:
        zone_status = "Yellow (Caution)"
        color = (0, 255, 255)
    else:
        zone_status = "Red (Danger)"
        color = (0, 0, 255)

    # Draw results on frame
    for box in results[0].boxes.data:
        x1, y1, x2, y2, conf, cls = map(int, box)
        if cls == 0:  # Only draw boxes for 'person'
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    # Display crowd status
    cv2.putText(frame, f"Zone: {zone_status} ({people_count} people)", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    cv2_imshow(frame) # Use cv2_imshow instead of cv2.imshow
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
!python yolo_detection.py

python3: can't open file '/content/yolo_detection.py': [Errno 2] No such file or directory


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
from datetime import datetime
from scipy.spatial import ConvexHull

# Load YOLOv8 model (Pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")

# Load the pre-recorded video
video_path = "/content/VID2.mp4"

if not os.path.exists(video_path):
    print(f"❌ Error: Video file '{video_path}' not found.")
    exit()

cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("❌ Error: Cannot open video file.")
    exit()

# Get input video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

if fps == 0 or frame_width == 0 or frame_height == 0:
    print("❌ Error: Could not retrieve video properties.")
    cap.release()
    exit()

# Set up output video writer
output_video_path = "output_dashboard.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width * 2, frame_height))

if not output_video.isOpened():
    print("❌ Error: VideoWriter failed to open!")
    cap.release()
    exit()

# Log file for alerts
alert_log_path = "alert_log.txt"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)

    # Extract person detections (bounding box centers)
    people_boxes = [
        ((int((box[0] + box[2]) / 2)), (int((box[1] + box[3]) / 2)))
        for box in results[0].boxes.data.cpu().numpy() if int(box[5]) == 0  # Class 0 = 'person'
    ]

    people_count = len(people_boxes)

    # Default area assumption for low counts
    occupied_area = 50000  # Default minimum area if fewer than 3 people detected

    if people_count >= 3:
        points = np.array(people_boxes)  # Convert to NumPy array
        hull = ConvexHull(points)  # Compute convex hull
        occupied_area = hull.volume  # Hull area (approximated)

    # Compute density (people per pixel squared)
    density = people_count / occupied_area if occupied_area > 0 else 0

    # Set density thresholds
    SAFE_DENSITY = 0.0005
    CAUTION_DENSITY = 0.0015
    DANGER_DENSITY = 0.0025

    # Determine zone status based on density
    if density < SAFE_DENSITY:
        zone_status = "Green (Safe)"
        color = (0, 255, 0)  # Green
        scatter_color = "green"
    elif density < CAUTION_DENSITY:
        zone_status = "Yellow (Caution)"
        color = (0, 255, 255)  # Yellow
        scatter_color = "yellow"
    else:
        zone_status = "Red (Danger)"
        color = (0, 0, 255)  # Red
        scatter_color = "red"

        # Log alert when danger zone is detected
        alert_message = f"🚨 ALERT: High crowd density detected! (People: {people_count}, Density: {density:.5f}) - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        print(alert_message)  # Print in console
        with open(alert_log_path, "a") as log_file:
            log_file.write(alert_message)

    # Draw bounding boxes on the frame
    for box in results[0].boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls = box.astype(int)
        if cls == 0:  # Only draw boxes for 'person'
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    # Display crowd status on video
    cv2.putText(frame, f"Zone: {zone_status} | Density: {density:.5f}", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    # Generate scatter plot as an image
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(0, frame.shape[1])  # X-axis based on frame width
    ax.set_ylim(frame.shape[0], 0)  # Y-axis based on frame height (inverted)
    ax.set_title(f"Crowd Density Map: {zone_status}")
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")

    # Plot people positions
    if people_boxes:
        x_vals, y_vals = zip(*people_boxes)
        ax.scatter(x_vals, y_vals, c=scatter_color, s=50, alpha=0.7)

    # Save scatter plot as an image
    scatter_path = "scatter_plot.jpg"
    plt.savefig(scatter_path, bbox_inches='tight')
    plt.close(fig)

    # Read scatter plot as an image
    scatter_img = cv2.imread(scatter_path)

    if scatter_img is None:
        print("❌ Error: Failed to load scatter plot image.")
        break

    scatter_img = cv2.resize(scatter_img, (frame_width, frame_height))

    # Combine video frame & scatter plot side by side
    combined_frame = np.hstack((frame, scatter_img))

    # Write frame to output video
    output_video.write(combined_frame)

cap.release()
output_video.release()
cv2.destroyAllWindows()

print(f"✅ Dashboard video saved at: {output_video_path}")
print(f"📄 Alerts logged in: {alert_log_path}")



0: 640x384 15 persons, 1 umbrella, 139.5ms
Speed: 4.4ms preprocess, 139.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 13 persons, 1 umbrella, 161.6ms
Speed: 4.7ms preprocess, 161.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 persons, 2 umbrellas, 171.7ms
Speed: 4.6ms preprocess, 171.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 15 persons, 2 umbrellas, 192.2ms
Speed: 4.7ms preprocess, 192.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 persons, 2 umbrellas, 159.1ms
Speed: 4.6ms preprocess, 159.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 persons, 2 umbrellas, 1 sports ball, 150.1ms
Speed: 4.6ms preprocess, 150.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 15 persons, 1 umbrella, 163.0ms
Speed: 4.7ms preprocess, 163.0ms inference, 1.3ms postprocess per image at shape (1, 

In [ ]:
from google.colab import files

video_path = "/content/output_dashboard.mp4"
files.download(video_path)


FileNotFoundError: Cannot find file: /content/output_dashboard.mp4

In [ ]:
import os
print(os.path.exists("/content/VID2.mp4"))  # Should return True


True
